In [29]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from dateutil.parser import parse
from itertools import compress
from collections import OrderedDict
from datetime import date
import datetime
import re
from datetime import timedelta
import openpyxl
from sklearn.metrics import mean_squared_error
import plotly.express as px

import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Create figure with secondary y-axis
import pathlib
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

In [30]:
#########################################
'''
Declare Functions
'''
#########################################
def convert_entire_DF_to_numeric(DF):
    for col in DF.columns:
        DF[col] = pd.to_numeric(DF[col], errors='coerce')
    return DF

def getTheLocalMinMax(df_col1):
    '''
    Code that computes takes the index of the dataframe where the values are either local minima or local maxima
    Parameter: df_col1 is a series whose index is datetime
    Returns a dictionary where the loc_min and loc_max are stored.
    '''
    
    df_col1 = pd.to_numeric(df_col1, errors='coerce')
    loc_min = (np.diff(np.sign(np.diff(df_col1))) > 0).nonzero()[0] + 1         # local min
    loc_max = (np.diff(np.sign(np.diff(df_col1))) < 0).nonzero()[0] + 1         # local max
    dict_ = {'loc_min': loc_min, 'loc_max': loc_max}

    return dict_

def computeInterval_peak_ave(pH_loc_min_series):
    '''
    Code that computes the duration between local minimum of the pH trend.
    Returns the average of the duration in minutes.
    
    Parameter: pH_loc_min_series is the pH reading corresponding to the local minima whose index is the DATETIME
    '''
    ser_TK03_pH_A = pH_loc_min_series
    time_diff_between_peaks = np.diff(ser_TK03_pH_A.index) # get the difference between the peaks in the pH reading series from Pi
    time_diff_bw_peaks_minutes = time_diff_between_peaks/np.timedelta64(1, 'm') # convert the units from nanoseconds to minutes
    
    return time_diff_bw_peaks_minutes.mean()



#used in the analysis results data. cleans up the sampling time entry
def time_cleaner(x):
    time_regex = re.compile(r'[0-8a-zA-Z:]', re.IGNORECASE| re.VERBOSE|re.DOTALL)
    timeact = time_regex.findall(x)
    newtime = ''.join(timeact)
    t = parse(x)
    new_str_time = str(t.hour)+':'+str(t.minute)+':'+str(t.second)
    return new_str_time

def generate_add_trace(figname, df_elemental, axis_bool):
    colname = df_elemental.columns[0]
    figname.add_trace(go.Scatter(x=df_elemental.index, y=df_elemental[colname], name=colname), secondary_y=axis_bool,)
#---------------------------------------


class LabResults():
    def __init__(self, monthyear, filePath):
        #____monthyear format examle is datetime.date(2020,7,1)
        #____filepath example is '\\thgobna001\userdata\THPAL\...\ANALYSIS RESULTS 2020\7) July _2020.xlsx'
        #____add r' at the start of the filepath to apply the regex that allows the use of the filepath as raw
        #____sef.workbook is the uploaded excel workbook of the entire analysis files so it takes time to load
        #____Create an instance of the workbook by giving it a varible name with the sample format: Jan_2020
        
        import openpyxl
        import io
        with open(filePath, "rb") as f:
            in_mem_file = io.BytesIO(f.read())
        print('.', end='')
        self.workbook = openpyxl.load_workbook(in_mem_file, data_only=True)
        self.monthyear = monthyear   
        print('.', end='')
        
    def generate_df_from_analysis(self, tabname, list_lab_index):
        #____tabname is a string type name of the tab in the laboratory analysis excel file.
        #____-------Example tabnames: '104PU01', '105TK03', '106TK01'
        #____-------Inside these tabs are the specific analysis results done on the sample taken from the 'tabname' area .
        #____list_lab_index is a list with the sample format is: ['B', 'C', 'E', '104_Pb']
        #____-------the first letter 'B' is the column letter where the day of the month is written as integers 1,2,3..30
        #____-------the second letter 'C' is the column letter where the sampling time is written; e.g. '7:00:00 AM'
        #____-------the third letter 'E' is the column where the results of the specific analysis are listed
        #____-------the last item in the list is the string you'll use as the new title; e.g. '104_Pb'
        print('.', end='')
        workbookTab = self.workbook[tabname]
        self.list_lab_index = list_lab_index
        col_sampleDate = self.list_lab_index[0]
        col_sampleTime = self.list_lab_index[1]
        col_analysis = self.list_lab_index[2]
        col_title = self.list_lab_index[3]
        currentmonthyr = self.monthyear
        
        #get the start and end sample dates
        Smpl_day_index = list()
        Chem_Element_datelist = list()
        Chem_Element_list = list()
        Smpl_day_index = list()
        Chem_Element_datelist = list()
        Chem_Element_list = list()
        plus_oneday_list = ['0:0:0', '1:0:0', '2:0:0', '3:0:0', '4:0:0', '5:0:0']
        #gets the date day from the B column of the analysis file if there is value in the cell
        for i in range(1, workbookTab.max_row):
            if type(workbookTab[str(col_sampleDate) + str(i)].value) == int or type(workbookTab[str(col_sampleDate) + str(i)].value) == float:
                Smpl_day_index.append(i)
        #get the ending row that has date day
        for j in range(1, workbookTab.max_row):
            if type(workbookTab[str(col_sampleTime) + str(j)].value) == datetime.datetime:
                smpl_end_index = j
        for y in range(0, len(Smpl_day_index)):
            if Smpl_day_index[y] != Smpl_day_index[-1]:      
                for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
                    if workbookTab[str(col_analysis)+str(x)].value is None: 
                        pass
                    else:
                        try:
                            if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                                date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                                +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                                Chem_Element_datelist.append(parse(date))  
                                chem_element = workbookTab[str(col_analysis)+str(x)].value
                                Chem_Element_list.append(chem_element)
                            else:
                                date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                                +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                                Chem_Element_datelist.append(parse(date))  
                                chem_element = workbookTab[str(col_analysis)+str(x)].value
                                Chem_Element_list.append(chem_element)
                        except:
                            pass

            if Smpl_day_index[y] == Smpl_day_index[-1]:
                for x in range(Smpl_day_index[y], smpl_end_index+1):
                    if workbookTab[str(col_analysis)+str(x)].value is None:   
                        pass
                    else:
                        try:
                            if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                                date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                                +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                                Chem_Element_datelist.append(parse(date))
                                chem_element = workbookTab[str(col_analysis)+str(x)].value
                                Chem_Element_list.append(chem_element)
                            else:
                                date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                                +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                                Chem_Element_datelist.append(parse(date))
                                chem_element = workbookTab[str(col_analysis)+str(x)].value
                                Chem_Element_list.append(chem_element)
                        except:
                            pass

        Chem_Element_list_pair = list(zip(Chem_Element_list, Chem_Element_datelist[0:]))
        Chem_Element_df = pd.DataFrame(Chem_Element_list_pair)
        Chem_Element_df.columns = [col_title, 'DATETIME'] ##check if list title is updated
        Chem_Element_df.set_index('DATETIME', inplace=True)
        print('.', end='')

        return Chem_Element_df
    
    
def create_blank_df_Limestone_lab_results():
    # Creates a blank dataframe with 21 columns.
    # The columns are the laboratory results for the three sampling points in limestone area: 111CY01, 111TK01, 111TK02
    # The index is DATETIME
    
    # Create a dictionary to pair up the column with the nan values
    blank_data_2 = {'DATETIME': np.nan, 'CY01_den': np.nan, 'CY01_prcnt_sol': np.nan, 
                    'CY01_ps10': np.nan, 'CY01_ps50': np.nan, 'CY01_ps90': np.nan,
                    'CY01_psMean': np.nan, 'CY01_ps_stdev': np.nan, 'TK01_den': np.nan, 
                    'TK01_prcnt_sol': np.nan, 'TK01_ps10': np.nan, 'TK01_ps50': np.nan, 
                    'TK01_ps90': np.nan, 'TK01_psMean': np.nan, 'TK01_ps_stdev': np.nan,
                    'TK02_den': np.nan, 'TK02_prcnt_sol': np.nan, 'TK02_ps10': np.nan,
                    'TK02_ps50': np.nan, 'TK02_ps90': np.nan, 'TK02_psMean': np.nan, 
                    'TK02_ps_stdev': np.nan}
    
    # Create the list of column names
    columnNames = ['DATETIME','CY01_den', 'CY01_prcnt_sol', 'CY01_ps10', 'CY01_ps50', 
                   'CY01_ps90', 'CY01_psMean', 'CY01_ps_stdev', 'TK01_den', 'TK01_prcnt_sol',
                   'TK01_ps10', 'TK01_ps50', 'TK01_ps90', 'TK01_psMean', 'TK01_ps_stdev',
                   'TK02_den', 'TK02_prcnt_sol', 'TK02_ps10', 'TK02_ps50', 'TK02_ps90',
                   'TK02_psMean', 'TK02_ps_stdev']
    
    # create the dataframe using pandas module. Then set the DATETIME as the index.
    df_LS = pd.DataFrame(blank_data_2, columns = columnNames)
    df_LS = df_LS.set_index('DATETIME')
    
    return df_LS
    

In [31]:
###############################
'''
Prepare the required variables for the class LabResults
'''
###############################
#filepath 2016 Laboratory Files
#------------------------------

filepath_01_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\01 JANUARY 2020\1) JANUARY_2020.xlsx'
filepath_02_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\02 FEBRUARY 2020\2) FEBRUARY _2020.xlsx'
filepath_03_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\03 MARCH 2020\3) March _2020.xlsx'
filepath_04_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\04 APRIL 2020\4) April _2020.xlsx'
filepath_05_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\05 MAY 2020\5) MAY_2020.xlsx'
filepath_06_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\06 JUNE 2020\6) June _2020.xlsx'
filepath_07_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\07 JULY 2020\7) July _2020.xlsx'
filepath_08_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\08 AUGUST 2020\8) August _2020.xlsx'
filepath_09_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\09 SEPTEMBER 2020\9) September _2020.xlsx'
filepath_10_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\10 OCTOBER 2020\10) October _2020.xlsx'
filepath_11_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\11 NOVEMBER 2020\11) November _2020.xlsx'

###############################
#datetime
#------------------------------
currentmonthyr_01_2020 = datetime.date(2020,1,1)
currentmonthyr_02_2020 = datetime.date(2020,2,1)
currentmonthyr_03_2020 = datetime.date(2020,3,1)
currentmonthyr_04_2020 = datetime.date(2020,4,1)
currentmonthyr_05_2020 = datetime.date(2020,5,1)
currentmonthyr_06_2020 = datetime.date(2020,6,1)
currentmonthyr_07_2020 = datetime.date(2020,7,1)
currentmonthyr_08_2020 = datetime.date(2020,8,1)
currentmonthyr_09_2020 = datetime.date(2020,9,1)
currentmonthyr_10_2020 = datetime.date(2020,10,1)
currentmonthyr_11_2020 = datetime.date(2020,11,1)

###############################
#List of datetime
#------------------------------
dateList2020 = [currentmonthyr_01_2020,currentmonthyr_02_2020,currentmonthyr_03_2020,currentmonthyr_04_2020,currentmonthyr_05_2020,
                currentmonthyr_06_2020,currentmonthyr_07_2020,currentmonthyr_08_2020,currentmonthyr_09_2020,currentmonthyr_10_2020,
                currentmonthyr_11_2020]

filepathList2020 = [filepath_01_2020,filepath_02_2020,filepath_03_2020,filepath_04_2020,filepath_05_2020,
                    filepath_06_2020,filepath_07_2020,filepath_08_2020,filepath_09_2020,filepath_10_2020,
                    filepath_11_2020]

###############################
#Variables containing text needed for the lab analysis
#------------------------------

LS_CY01_density = ['B', 'C', 'AI', 'CY01_den']
LS_CY01_prcnt_solid = ['B', 'C', 'AJ', 'CY01_prcnt_sol']
LS_CY01_ps_10 = ['B', 'C', 'AO', 'CY01_ps10']
LS_CY01_ps_50 = ['B', 'C', 'AP', 'CY01_ps50']
LS_CY01_ps_90 = ['B', 'C', 'AQ', 'CY01_ps90']
LS_CY01_ps_mean = ['B', 'C', 'AR', 'CY01_psMean']
LS_CY01_ps_stdev_mean = ['B', 'C', 'AS', 'CY01_ps_stdev']

LS_TK01_density = ['B', 'C', 'AI', 'TK01_den']
LS_TK01_prcnt_solid = ['B', 'C', 'AJ', 'TK01_prcnt_sol']
LS_TK01_ps_10 = ['B', 'C', 'AO', 'TK01_ps10']
LS_TK01_ps_50 = ['B', 'C', 'AP', 'TK01_ps50']
LS_TK01_ps_90 = ['B', 'C', 'AQ', 'TK01_ps90']
LS_TK01_ps_mean = ['B', 'C', 'AR', 'TK01_psMean']
LS_TK01_ps_stdev_mean = ['B', 'C', 'AS', 'TK01_ps_stdev']

LS_TK02_density = ['B', 'C', 'AI', 'TK02_den']
LS_TK02_prcnt_solid = ['B', 'C', 'AJ', 'TK02_prcnt_sol']
LS_TK02_ps_10 = ['B', 'C', 'AO', 'TK02_ps10']
LS_TK02_ps_50 = ['B', 'C', 'AP', 'TK02_ps50']
LS_TK02_ps_90 = ['B', 'C', 'AQ', 'TK02_ps90']
LS_TK02_ps_mean = ['B', 'C', 'AR', 'TK02_psMean']
LS_TK02_ps_stdev_mean = ['B', 'C', 'AS', 'TK02_ps_stdev']


In [32]:
##################
'''
Load the Pi data
'''
##################
raw_Pi_df = pd.read_excel(r'C:\Users\v.t.flores\Documents\BallMill_Grinding_Eval_Pi.xlsx') #Load the Pidata from the excel file
raw_Pi_df = raw_Pi_df.set_index('DATETIME')
raw_Pi_df = convert_entire_DF_to_numeric(raw_Pi_df)[1:] 


LS_Pi = raw_Pi_df # store to another variable name
LS_Pi.columns = ['BC02_tonnage', 'BC05_tonnage', 'Mill_Curr', 'Dilution_Mill', 'Dilution_TK01', 'Dilution_Total'] #rename the columns
LS_Pi = LS_Pi[1:] # select the second row onwards

# compute the total feed flowrate. Add the bc05 tonnage, the water dilution to ballMill, and the circulating load which is estimated to be 120% of the limestone feed
LS_Pi['feed_slurry_total_bc05Based'] = LS_Pi['BC05_tonnage'] + LS_Pi['Dilution_Mill'] + LS_Pi['BC05_tonnage']*1.2 
LS_Pi['RT_bc05based_CL'] = 133/LS_Pi['feed_slurry_total_bc05Based'] # compute the retention time based on the effective volume of ballmill= 133 m3 and the total input flowrate to the mill
LS_Pi = LS_Pi[LS_Pi['BC02_tonnage']>10] # set the minimum bc02 tonnage
LS_Pi = LS_Pi[LS_Pi['BC05_tonnage']>10] # set the minimum bc05 tonnage

LS_Pi['Dilution_TK01'] = np.where(LS_Pi['Dilution_TK01']<0, 0, LS_Pi['Dilution_TK01']) # remove the negative values for the dilution flowrate

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [10]:
#GATHER THE LABORATORY RESULTS FOR LIMESTONE AREA

df_LS = create_blank_df_Limestone_lab_results() #use the blank dataframe as the initial dataframe where the monthly labresults dataframes will append to.
for i in range(len(dateList2020)):
    
    monthLab = LabResults(dateList2020[i], filepathList2020[i])
    df_month = monthLab.generate_df_from_analysis('111CY01', LS_CY01_density).join(
        monthLab.generate_df_from_analysis('111CY01', LS_CY01_prcnt_solid), how='outer').join(
        monthLab.generate_df_from_analysis('111CY01', LS_CY01_ps_10), how='outer').join(
        monthLab.generate_df_from_analysis('111CY01', LS_CY01_ps_50), how='outer').join(
        monthLab.generate_df_from_analysis('111CY01', LS_CY01_ps_90), how='outer').join(
        monthLab.generate_df_from_analysis('111CY01', LS_CY01_ps_mean), how='outer').join(
        monthLab.generate_df_from_analysis('111CY01', LS_CY01_ps_stdev_mean), how='outer').join(
        monthLab.generate_df_from_analysis('111TK01', LS_TK01_density), how='outer').join(
        monthLab.generate_df_from_analysis('111TK01', LS_TK01_prcnt_solid), how='outer').join(
        monthLab.generate_df_from_analysis('111TK01', LS_TK01_ps_10), how='outer').join(
        monthLab.generate_df_from_analysis('111TK01', LS_TK01_ps_50), how='outer').join(
        monthLab.generate_df_from_analysis('111TK01', LS_TK01_ps_90), how='outer').join(
        monthLab.generate_df_from_analysis('111TK01', LS_TK01_ps_mean), how='outer').join(
        monthLab.generate_df_from_analysis('111TK01', LS_TK01_ps_stdev_mean), how='outer').join(
        monthLab.generate_df_from_analysis('111TK02', LS_TK02_density), how='outer').join(
        monthLab.generate_df_from_analysis('111TK02', LS_TK02_prcnt_solid), how='outer').join(
        monthLab.generate_df_from_analysis('111TK02', LS_TK02_ps_10), how='outer').join(
        monthLab.generate_df_from_analysis('111TK02', LS_TK02_ps_50), how='outer').join(
        monthLab.generate_df_from_analysis('111TK02', LS_TK02_ps_90), how='outer').join(
        monthLab.generate_df_from_analysis('111TK02', LS_TK02_ps_mean), how='outer').join(
        monthLab.generate_df_from_analysis('111TK02', LS_TK02_ps_stdev_mean), how='outer')

    print(i)
    df_LS = df_LS.append(df_month, sort=True)


#-------------------------------------

..........................................0
..........................................1
..........................................2
..........................................3
..........................................4
..........................................5
..........................................6
..........................................7
..........................................8
..........................................9
..........................................10


In [68]:
print('min RT: {}'.format(LS_Pi['RT_bc05based_CL'].min()))
print('max RT: {}'.format(LS_Pi['RT_bc05based_CL'].max()))

min RT: 0.2763382904689249
max RT: 4.421673285132071


In [148]:
LS_Pi_above_100m3h['RT_bc05based_CL'].mean()

0.3986351346455376

#### above 100tph

In [159]:
#create a df of lab results with adjusted sampling time. adjusted by 1 hr less.
df_LS_RT_adj = df_LS.reset_index()
df_LS_RT_adj['DATETIME'] = df_LS_RT_adj['DATETIME'] - datetime.timedelta(hours=1)
df_LS_RT_adj = df_LS_RT_adj.set_index('DATETIME')

LS_Pi_above_100m3h = LS_Pi[LS_Pi['BC05_tonnage']>100]
LS_Pi_above_100m3h = LS_Pi_above_100m3h[LS_Pi_above_100m3h['RT_bc05based_CL']>0.3]
LS_Pi_above_100m3h = LS_Pi_above_100m3h[LS_Pi_above_100m3h['RT_bc05based_CL']<=0.4]
LS_Pi_above_100m3h = LS_Pi_above_100m3h[LS_Pi_above_100m3h['BC05_tonnage']<=120]
LS_Pi_above_100m3h = LS_Pi_above_100m3h[LS_Pi_above_100m3h['Dilution_Total']>200]
LS_Pi_above_100m3h = LS_Pi_above_100m3h[LS_Pi_above_100m3h['Dilution_Total']<=230]

#Adjust the datetime by the retention time then match the lab analysis for that time.
'''
for item in range(len(LS_Pi_above_100m3h)):
    Dtime = LS_Pi_above_100m3h.index[item]
    #print(Dtime)
    Dtime_plus2 = Dtime + datetime.timedelta(hours=2)
    # labtime > dtime but less than dtime + 2hrs
    for labtime in df_LS_RT_adj.index:
        if (labtime > Dtime) and (labtime <= Dtime_plus2):
            print(labtime)
'''

def find_the_appro_labTime(df_pi, df_lab):
    labtime_list = []
    for item in range(len(df_pi)):
        Dtime = df_pi.index[item]
        #print(Dtime)
        Dtime_plus2 = Dtime + datetime.timedelta(hours=2)
        # labtime > dtime but less than dtime + 2hrs
        for labtime in df_lab.index:
            if (labtime > Dtime) and (labtime <= Dtime_plus2):
                print(labtime)
                labtime_list.append(labtime)
                
    return labtime_list


labtime_list = find_the_appro_labTime(LS_Pi_above_100m3h, df_LS_RT_adj)

blank_data_2 = {'DATETIME': np.nan, 'CY01_den': np.nan, 'CY01_prcnt_sol': np.nan, 'CY01_ps10': np.nan, 'CY01_ps50': np.nan, 'CY01_ps90': np.nan,
       'CY01_psMean': np.nan, 'CY01_ps_stdev': np.nan, 'TK01_den': np.nan, 'TK01_prcnt_sol': np.nan,
       'TK01_ps10': np.nan, 'TK01_ps50': np.nan, 'TK01_ps90': np.nan, 'TK01_psMean': np.nan, 'TK01_ps_stdev': np.nan,
       'TK02_den': np.nan, 'TK02_prcnt_sol': np.nan, 'TK02_ps10': np.nan, 'TK02_ps50': np.nan, 'TK02_ps90': np.nan,
       'TK02_psMean': np.nan, 'TK02_ps_stdev': np.nan}
columnNames = ['DATETIME','CY01_den', 'CY01_prcnt_sol', 'CY01_ps10', 'CY01_ps50', 'CY01_ps90',
       'CY01_psMean', 'CY01_ps_stdev', 'TK01_den', 'TK01_prcnt_sol',
       'TK01_ps10', 'TK01_ps50', 'TK01_ps90', 'TK01_psMean', 'TK01_ps_stdev',
       'TK02_den', 'TK02_prcnt_sol', 'TK02_ps10', 'TK02_ps50', 'TK02_ps90',
       'TK02_psMean', 'TK02_ps_stdev']
df_LS_temp = pd.DataFrame(blank_data_2, columns = columnNames)
df_LS_temp = df_LS_temp.set_index('DATETIME')

for item in labtime_list:
    print(item)
    df_LS_temp = df_LS_temp.append(df_LS_RT_adj.loc[item:item])
    
df_LS_temp_dropped_dup = df_LS_temp.drop_duplicates(keep="first") 


# df_LS_temp_dropped_dup. This dataframe contains the data where the appropriate samples for the LS_Pi_above_100m3h listed
df_LS_temp_dropped_dup[['TK01_ps10', 'TK01_ps50', 'TK01_ps90','TK01_psMean', 'TK01_ps_stdev']]

In [223]:
#Create a function to adjust the dataframe by the retention time
def adjust_DATEIME_by_RT(df_Pi):
    '''
    df_Pi ==> this is the dataframe containing the limestone area pi data.
    what it does:
        adds the retention time to the datetime.
        returns the same dataframe after adjusting the datetime.
    '''
    
    df_Pi = df_Pi.reset_index() # reset the index to free up the DATETIME column
    df_Pi['RT_tDelta'] = [datetime.timedelta(hours=x) for x in df_Pi['RT_bc05based_CL']] #create a column for retention time converted to timedelta

    #add the RT_delta to the DATETIME column. this is the actual time when feed to the ball mill comes out of the mill
    df_Pi['DATETIME'] = df_Pi['DATETIME'] + df_Pi['RT_tDelta'] 
    df_Pi = df_Pi.set_index('DATETIME')
    
    return df_Pi

def find_the_appro_labTime(df_pi, df_lab):
    labtime_list = []
    for item in range(len(df_pi)):
        Dtime = df_pi.index[item]
        #print(Dtime)
        Dtime_plus2 = Dtime + datetime.timedelta(hours=2)
        # labtime > dtime but less than dtime + 2hrs
        for labtime in df_lab.index:
            if (labtime > Dtime) and (labtime <= Dtime_plus2):
                print('.', end='')
                labtime_list.append(labtime)
                
    return labtime_list

def create_df_of_pi_lab_of_same_tonnage_and_rt(df_LS, LS_Pi, tonnage, ret_time, dilution): 
    '''
    df_LS ==> the dataframe for the laboratory results of limestone area from the sampling points: CY01, TK01, TK02
    LS_Pi ==> the dataframe for the Pi data for the parameters in limestone area: BC02 WIC, BC05 WIC, FIC003, FIC006, MILL CURRENT etc (see. LS_Pi.columns)
    
    Selects only the 111tk01 particle size analysis results
    returns the lab results pointing to the time where the Pi data parameters packaged as a dataframe
    '''
    #ADjust the datetime column by the retention time
    LS_Pi = adjust_DATEIME_by_RT(LS_Pi)
    
    #create a df of lab results with adjusted sampling time. adjusted by 1 hr less.
    df_LS_RT_adj = df_LS.reset_index()
    df_LS_RT_adj['DATETIME'] = df_LS_RT_adj['DATETIME'] - datetime.timedelta(hours=1)
    df_LS_RT_adj = df_LS_RT_adj.set_index('DATETIME')

    LS_Pi_above_100m3h = LS_Pi[LS_Pi['BC05_tonnage']>tonnage['min']]
    LS_Pi_above_100m3h = LS_Pi_above_100m3h[LS_Pi_above_100m3h['RT_bc05based_CL']>ret_time['min']]
    LS_Pi_above_100m3h = LS_Pi_above_100m3h[LS_Pi_above_100m3h['RT_bc05based_CL']<=ret_time['max']]
    LS_Pi_above_100m3h = LS_Pi_above_100m3h[LS_Pi_above_100m3h['BC05_tonnage']<=tonnage['max']]
    LS_Pi_above_100m3h = LS_Pi_above_100m3h[LS_Pi_above_100m3h['Dilution_Total']>dilution['min']]
    LS_Pi_above_100m3h = LS_Pi_above_100m3h[LS_Pi_above_100m3h['Dilution_Total']<=dilution['max']]

    #Adjust the datetime by the retention time then match the lab analysis for that time.
    
    '''
    ##### reference: what the find_appro_labTime function does
    def find_the_appro_labTime(df_pi, df_lab):
        labtime_list = []
        for item in range(len(df_pi)):
            Dtime = df_pi.index[item]
            #print(Dtime)
            Dtime_plus2 = Dtime + datetime.timedelta(hours=2)
            # labtime > dtime but less than dtime + 2hrs
            for labtime in df_lab.index:
                if (labtime > Dtime) and (labtime <= Dtime_plus2):
                    print(labtime)
                    labtime_list.append(labtime)

        return labtime_list
    '''
    labtime_list = find_the_appro_labTime(LS_Pi_above_100m3h, df_LS_RT_adj)

    blank_data_2 = {'DATETIME': np.nan, 'CY01_den': np.nan, 'CY01_prcnt_sol': np.nan, 'CY01_ps10': np.nan, 
                    'CY01_ps50': np.nan, 'CY01_ps90': np.nan, 'CY01_psMean': np.nan, 'CY01_ps_stdev': np.nan, 
                    'TK01_den': np.nan, 'TK01_prcnt_sol': np.nan, 'TK01_ps10': np.nan, 'TK01_ps50': np.nan, 'TK01_ps90': np.nan, 
                    'TK01_psMean': np.nan, 'TK01_ps_stdev': np.nan, 'TK02_den': np.nan, 'TK02_prcnt_sol': np.nan, 'TK02_ps10': np.nan, 
                    'TK02_ps50': np.nan, 'TK02_ps90': np.nan, 'TK02_psMean': np.nan, 'TK02_ps_stdev': np.nan}
    columnNames = ['DATETIME','CY01_den', 'CY01_prcnt_sol', 'CY01_ps10', 'CY01_ps50', 
                   'CY01_ps90', 'CY01_psMean', 'CY01_ps_stdev', 'TK01_den', 'TK01_prcnt_sol',
                   'TK01_ps10', 'TK01_ps50', 'TK01_ps90', 'TK01_psMean', 'TK01_ps_stdev',
                   'TK02_den', 'TK02_prcnt_sol', 'TK02_ps10', 'TK02_ps50', 'TK02_ps90',
                   'TK02_psMean', 'TK02_ps_stdev']
    df_LS_temp = pd.DataFrame(blank_data_2, columns = columnNames)
    df_LS_temp = df_LS_temp.set_index('DATETIME')

    for item in labtime_list:
        print('.', end='')
        df_LS_temp = df_LS_temp.append(df_LS_RT_adj.loc[item:item])

    df_LS_temp_dropped_dup = df_LS_temp.drop_duplicates(keep="first") 


    # df_LS_temp_dropped_dup. This dataframe contains the data where the appropriate samples for the LS_Pi_above_100m3h listed
    df_return = df_LS_temp_dropped_dup[['TK01_ps10', 'TK01_ps50', 'TK01_ps90','TK01_psMean', 'TK01_ps_stdev']]
    df_return['tonnage_max'] = tonnage['max']
    df_return['tonnage_min'] = tonnage['min']
    
    df_return['retT_max'] = ret_time['max']
    df_return['retT_min'] = ret_time['min']
    
    df_return['dilution_max'] = dilution['max']
    df_return['dilution_min'] = dilution['min']
    
    return df_return

In [217]:

# Set-up the limits for relevant parameters
tonnage = {'max':120, 'min': 100}
ret_time = {'max':0.4, 'min': 0.3}
dilution = {'max':230, 'min': 200}

df_lab_appropriate = create_df_of_pi_lab_of_same_tonnage_and_rt(df_LS, LS_Pi, tonnage, ret_time, dilution)


................

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

In [218]:
df_lab_appropriate.dropna(how='any').to_excel(r'C:\Users\v.t.flores\Documents\BallMill_proj_lab_results_range1a.xlsx')


In [243]:
# Set-up the limits for relevant parameters
tonnage = {'max':120, 'min': 90}
ret_time = {'max':0.5, 'min': 0.3}
dilution = {'max':250, 'min': 100}

df_lab_appropriate = create_df_of_pi_lab_of_same_tonnage_and_rt(df_LS, LS_Pi, tonnage, ret_time, dilution)
df_lab_appropriate.dropna(how='any').to_excel(r'C:\Users\v.t.flores\Documents\BallMill_proj_lab_results_range_5.xlsx')


......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

In [244]:
df_lab_appropriate

,TK01_ps10,TK01_ps50,TK01_ps90,TK01_psMean,TK01_ps_stdev,tonnage_max,tonnage_min,retT_max,retT_min,dilution_max,dilution_min
DATETIME,,,,,,,,,,,
2020-01-05 09:00:00,2.4159,4.5892,6.7055,4.50797,1.9116,120,90,0.5,0.3,250,100
2020-01-09 01:00:00,3.5527,6.1108,9.8347,6.46181,2.5342,120,90,0.5,0.3,250,100
2020-01-10 01:00:00,3.9020,9.9164,27.0079,14.25507,15.9336,120,90,0.5,0.3,250,100
2020-01-12 01:00:00,3.8304,9.6218,23.1226,12.49229,11.0318,120,90,0.5,0.3,250,100
2020-01-16 01:00:00,3.6763,8.5353,17.0391,9.88091,6.5404,120,90,0.5,0.3,250,100
...,...,...,...,...,...,...,...,...,...,...,...
2020-11-11 01:00:00,NaN,NaN,NaN,NaN,NaN,120,90,0.5,0.3,250,100
2020-11-14 09:00:00,2.4272,4.1972,6.5458,4.37717,1.6338,120,90,0.5,0.3,250,100
2020-11-15 01:00:00,NaN,NaN,NaN,NaN,NaN,120,90,0.5,0.3,250,100


In [233]:
df_LS[['TK01_ps10', 'TK01_ps50', 'TK01_ps90','TK01_psMean', 'TK01_ps_stdev']].dropna(how='any').to_excel(r'C:\Users\v.t.flores\Documents\BallMill_proj_entire_labres.xlsx')

In [17]:
df_subset_pi = LS_Pi[['BC05_tonnage','Mill_Curr', 'RT_bc05based_CL']]
df_subset_pi = df_subset_pi[df_subset_pi['BC05_tonnage']>90]
df_subset_pi = df_subset_pi[df_subset_pi['BC05_tonnage']<=120]
#df_subset_pi = df_subset_pi[df_subset_pi['RT_bc05based_CL']<=0.5]
#df_subset_pi = df_subset_pi[df_subset_pi['RT_bc05based_CL']>0.3]
#df_subset_pi.to_excel(r'C:\Users\v.t.flores\Documents\BallMill_proj_pi_millcurrent_d.xlsx')

In [26]:
df_subset_pi_90to100 = df_subset_pi[(df_subset_pi['BC05_tonnage']>90) & (df_subset_pi['BC05_tonnage']<=100)]
df_subset_pi_100to110 = df_subset_pi[(df_subset_pi['BC05_tonnage']>100) & (df_subset_pi['BC05_tonnage']<=120)]
df_subset_pi_above120 = df_subset_pi[df_subset_pi['BC05_tonnage']>120]

lab_forplot_111tk01 = df_LS[['TK01_ps10', 'TK01_ps50', 'TK01_ps90','TK01_psMean', 'TK01_ps_stdev']]

FIG2 = make_subplots(specs=[[{"secondary_y": True}]])
FIG2.add_trace(go.Scatter(x = df_subset_pi_100to110.index, y = df_subset_pi_100to110['RT_bc05based_CL'],name='RT_bc05based_CL'))
FIG2.add_trace(go.Scatter(x = df_subset_pi_100to110.index, y = df_subset_pi_100to110['Mill_Curr'],name='Mill_Curr'), secondary_y=True)
FIG2.add_trace(go.Scatter(x = df_subset_pi_100to110.index, y = df_subset_pi_100to110['BC05_tonnage'],name='BC05_tonnage'), secondary_y=True)
FIG2.update_layout()
FIG2.update_yaxes(title_text="rt_hours", secondary_y=False) # Set y-axes titles primary axis
FIG2.update_yaxes(title_text="current", secondary_y=True) # Set y-axes titles secondary axis
FIG2.update_xaxes(title_text="DateTime")


In [28]:
FIG2 = make_subplots(specs=[[{"secondary_y": True}]])
FIG2.add_trace(go.Scatter(x = df_subset_pi_100to110['Mill_Curr'], y = df_subset_pi_100to110['RT_bc05based_CL'],name='RT_bc05based_CL',  mode='markers'))
#FIG2.add_trace(go.Scatter(x = df_subset_pi_100to110.index, y = df_subset_pi_100to110['Mill_Curr'],name='Mill_Curr',  mode='markers'), secondary_y=True)
#FIG2.add_trace(go.Scatter(x = df_subset_pi_100to110.index, y = df_subset_pi_100to110['BC05_tonnage'],name='BC05_tonnage',  mode='markers'), secondary_y=True)
#FIG2.update_layout()
FIG2.update_yaxes(title_text="rt_hours", secondary_y=False) # Set y-axes titles primary axis
#FIG2.update_yaxes(title_text="current", secondary_y=True) # Set y-axes titles secondary axis
FIG2.update_xaxes(title_text="current")

In [24]:
df_subset_pi_above120

,BC05_tonnage,Mill_Curr,RT_bc05based_CL
DATETIME,,,


In [242]:
LS_Pi[['BC05_tonnage','Mill_Curr', 'RT_bc05based_CL']].to_excel(r'C:\Users\v.t.flores\Documents\BallMill_proj_pi_millcurrent_e.xlsx')

In [254]:
# Grouping the data by weekly frequency and then applying the aggregate function. This is done by the function 'resample'

#df_subset_pi.resample('M', on='week_starting').mean()
df_subset_pi.resample('W').mean().to_excel(r'C:\Users\v.t.flores\Documents\BallMill_proj_Pi_parameters_weekly_mean.xlsx')
df_lab_appropriate.resample('W').max()['TK01_psMean'].to_excel(r'C:\Users\v.t.flores\Documents\BallMill_proj_particleSize_weekly.xlsx')

In [255]:
df_subset_pi_LS = df_subset_pi.resample('W').mean()
df_lab_appropriate_LS = df_lab_appropriate.resample('W').max()['TK01_psMean']

In [14]:
df_LS.columns

Index(['CY01_den', 'CY01_prcnt_sol', 'CY01_ps10', 'CY01_ps50', 'CY01_ps90',
       'CY01_psMean', 'CY01_ps_stdev', 'TK01_den', 'TK01_prcnt_sol',
       'TK01_ps10', 'TK01_ps50', 'TK01_ps90', 'TK01_psMean', 'TK01_ps_stdev',
       'TK02_den', 'TK02_prcnt_sol', 'TK02_ps10', 'TK02_ps50', 'TK02_ps90',
       'TK02_psMean', 'TK02_ps_stdev'],
      dtype='object')

In [16]:
lab_forplot_111tk01 = df_LS[['TK01_ps10', 'TK01_ps50', 'TK01_ps90','TK01_psMean', 'TK01_ps_stdev']]

FIG1 = make_subplots(specs=[[{"secondary_y": True}]])
FIG1.add_trace(go.Scatter(x = df_LS.index, y = df_LS['TK01_psMean'],name='TK01_psMean'))
FIG1.add_trace(go.Scatter(x = df_LS.index, y = df_LS['TK02_psMean'],name='TK02_psMean'), secondary_y=True)
FIG1.update_layout()
FIG1.update_yaxes(title_text="microns", secondary_y=False) # Set y-axes titles primary axis
FIG1.update_xaxes(title_text="DateTime")


#### RT 30 mins to 1hr 70 to 80 tph

In [97]:
#know data 30 mins to 1 hr range RT
LS_Pi_rt_30_to_1hr = LS_Pi[LS_Pi['RT_bc05based_CL']>0.5]
LS_Pi_rt_30_to_1hr = LS_Pi_rt_30_to_1hr[LS_Pi_rt_30_to_1hr['RT_bc05based_CL']<=1]
LS_Pi_rt_30_to_1hr_adj = LS_Pi_rt_30_to_1hr.reset_index() # reset the index to free up the DATETIME column
LS_Pi_rt_30_to_1hr_adj['RT_tDelta'] = [datetime.timedelta(hours=x) for x in LS_Pi_rt_30_to_1hr_adj['RT_bc05based_CL']] #create a column for retention time converted to timedelta

#add the RT_delta to the DATETIME column. this is the actual time when feed to the ball mill comes out of the mill
LS_Pi_rt_30_to_1hr_adj['DATETIME'] = LS_Pi_rt_30_to_1hr_adj['DATETIME'] + LS_Pi_rt_30_to_1hr_adj['RT_tDelta'] 
LS_Pi_rt_30_to_1hr_adj = LS_Pi_rt_30_to_1hr_adj.set_index('DATETIME')

def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)+timedelta(hours=t.minute//30))

#round the adjusted DATETIME to the nearest hour
LS_Pi_rt_30_to_1hr_adj['DATETIME'] = LS_Pi_rt_30_to_1hr_adj['DATETIME'].apply(lambda x: hour_rounder((x)))
#LS_Pi_rt_30_to_1hr_adj['DATETIME'] = [hour_rounder(x) for x in LS_Pi_rt_30_to_1hr_adj['DATETIME']] ##gives the same result

df_LS_30_to_1hr_adj = df_LS.reset_index()
df_LS_30_to_1hr_adj['DATETIME'] = df_LS_30_to_1hr_adj['DATETIME'] - datetime.timedelta(hours=1)
df_LS_30_to_1hr_adj = df_LS_30_to_1hr_adj.set_index('DATETIME')
#Join the Pi and lab dataframes
LS_Pi_lab_30_to_1hr = LS_Pi_rt_30_to_1hr_adj.join(df_LS_30_to_1hr_adj, how='outer')
LS_Pi_lab_30_to_1hr_noNAN = LS_Pi_lab_30_to_1hr.dropna(how='any')
#Know the data for the range 70 to 80 tph of bc05
LS_Pi_lab_30_to_1hr_noNAN1 = LS_Pi_lab_30_to_1hr_noNAN[LS_Pi_lab_30_to_1hr_noNAN['BC05_tonnage']>70]
LS_Pi_lab_30_to_1hr_noNAN1 = LS_Pi_lab_30_to_1hr_noNAN1[LS_Pi_lab_30_to_1hr_noNAN1['BC05_tonnage']<=80]


#### RT 30 mins to 1hr 80 to 90tph

In [125]:
#know data 30 mins to 1 hr range RT
LS_Pi_rt_30_to_1hr_2 = LS_Pi[LS_Pi['RT_bc05based_CL']>0.5]
LS_Pi_rt_30_to_1hr_2 = LS_Pi_rt_30_to_1hr_2[LS_Pi_rt_30_to_1hr_2['RT_bc05based_CL']<=1]
LS_Pi_rt_30_to_1hr_2_adj = LS_Pi_rt_30_to_1hr.reset_index()
LS_Pi_rt_30_to_1hr_2_adj['RT_tDelta'] = [datetime.timedelta(hours=x) for x in LS_Pi_rt_30_to_1hr_2_adj['RT_bc05based_CL']]
# add the RT to the DATETIME
LS_Pi_rt_30_to_1hr_2_adj['DATETIME'] = LS_Pi_rt_30_to_1hr_2_adj['DATETIME'] + LS_Pi_rt_30_to_1hr_2_adj['RT_tDelta']
#LS_Pi_rt_30_to_1hr_adj = LS_Pi_rt_30_to_1hr_adj.set_index('DATETIME')

def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)+timedelta(hours=t.minute//30))

#round the adjusted DATETIME to the nearest hour
LS_Pi_rt_30_to_1hr_2_adj['DATETIME'] = LS_Pi_rt_30_to_1hr_2_adj['DATETIME'].apply(lambda x: hour_rounder((x)))
#LS_Pi_rt_30_to_1hr_adj['DATETIME'] = [hour_rounder(x) for x in LS_Pi_rt_30_to_1hr_adj['DATETIME']] ##gives the same result
LS_Pi_rt_30_to_1hr_2_adj = LS_Pi_rt_30_to_1hr_2_adj.set_index('DATETIME')

In [138]:
df_LS_30_to_1hr_adj_2 = df_LS.reset_index()
df_LS_30_to_1hr_adj_2['DATETIME'] = df_LS_30_to_1hr_adj_2['DATETIME'] - datetime.timedelta(hours=1)
df_LS_30_to_1hr_adj_2 = df_LS_30_to_1hr_adj_2.set_index('DATETIME')
LS_Pi_lab_30_to_1hr_2 = LS_Pi_rt_30_to_1hr_2_adj.join(df_LS_30_to_1hr_adj_2, how='outer')
LS_Pi_lab_30_to_1hr_noNAN_2 = LS_Pi_lab_30_to_1hr_2.dropna(how='any')

#Know the data for the range 70 to 80 tph of bc05
LS_Pi_lab_30_to_1hr_noNAN_2 = LS_Pi_lab_30_to_1hr_noNAN_2[LS_Pi_lab_30_to_1hr_noNAN_2['BC05_tonnage']>80]
LS_Pi_lab_30_to_1hr_noNAN_2 = LS_Pi_lab_30_to_1hr_noNAN_2[LS_Pi_lab_30_to_1hr_noNAN_2['BC05_tonnage']<=100]

In [141]:
LS_Pi_lab_30_to_1hr_noNAN_2.columns

Index(['BC02_tonnage', 'BC05_tonnage', 'Mill_Curr', 'Dilution_Mill',
       'Dilution_TK01', 'Dilution_Total', 'feed_slurry_total_bc05Based',
       'RT_bc05based_CL', 'RT_tDelta', 'CY01_den', 'CY01_prcnt_sol',
       'CY01_ps10', 'CY01_ps50', 'CY01_ps90', 'CY01_psMean', 'CY01_ps_stdev',
       'TK01_den', 'TK01_prcnt_sol', 'TK01_ps10', 'TK01_ps50', 'TK01_ps90',
       'TK01_psMean', 'TK01_ps_stdev', 'TK02_den', 'TK02_prcnt_sol',
       'TK02_ps10', 'TK02_ps50', 'TK02_ps90', 'TK02_psMean', 'TK02_ps_stdev'],
      dtype='object')

In [140]:
#export to excel then chart the data there
LS_Pi_lab_30_to_1hr_noNAN_2[['CY01_ps10', 'CY01_ps50', 'CY01_ps90', 'CY01_psMean', 'CY01_ps_stdev']].to_excel(r'C:\Users\v.t.flores\Documents\Ball_mill_CY01_30_1hrRT_2.xlsx')

In [142]:
LS_Pi_lab_30_to_1hr_noNAN1[['TK01_ps10', 'TK01_ps50', 'TK01_ps90','TK01_psMean', 'TK01_ps_stdev']].to_excel(r'C:\Users\v.t.flores\Documents\Ball_mill_TK01_30_1hrRT_1.xlsx')
LS_Pi_lab_30_to_1hr_noNAN_2[['TK01_ps10', 'TK01_ps50', 'TK01_ps90','TK01_psMean', 'TK01_ps_stdev']].to_excel(r'C:\Users\v.t.flores\Documents\Ball_mill_TK01_30_1hrRT_2.xlsx')

In [95]:
#export to excel then chart the data there
LS_Pi_lab_30_to_1hr_noNAN1[['CY01_ps10', 'CY01_ps50', 'CY01_ps90', 'CY01_psMean', 'CY01_ps_stdev']].to_excel(r'C:\Users\v.t.flores\Documents\Ball_mill_CY01_30_1hrRT.xlsx')